In [2]:
import os 
path = "/home/mr/Grive/01.ALDA/Project-Alda/data"
import math
import numpy as np

In [18]:
ratings = sc.textFile(os.path.join(path, 'ratings.csv')) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1) \
    .map(lambda a: (a[0], a[1], a[2]))

header = ratings.first()
ratings = ratings.filter(lambda line: line != header)
ratings = ratings.map(lambda x:(int(x[0]), int(x[1]), float(x[2])))
ratings = ratings.sample(False, 0.5, seed = 13)
ratings2 = ratings.map(lambda x: ((x[0], x[1]), x[2]))
user_ratings = ratings.map(lambda x:(x[0], x[2])).groupByKey()
user_ratings_mean_std = user_ratings.map(lambda x:(x[0], (np.mean(list(x[1])), np.std(list(x[1])))))
normalized_ratings = ratings.map(lambda x:(x[0], (x[1], x[2])))\
                                .join(user_ratings_mean_std)\
                                .map(lambda x:(x[0], x[1][0][0], (x[1][0][1]-x[1][1][0])/x[1][1][1]))
        

In [19]:
seed = 133
trainData, testData = normalized_ratings.randomSplit([0.6, 0.4], seed)

In [20]:
trainData.cache()
normalized_ratings.count()



50091

In [21]:
movie_user = normalized_ratings.map(lambda x:(x[1], x[0])).groupByKey()
user_movie = normalized_ratings.map(lambda x:(x[0], x[1])).groupByKey()
user_movie_rating = normalized_ratings.map(lambda x:((x[0], x[1]), x[2]))

In [23]:
user_count = user_movie.count()

671
7059


In [ ]:
dist_matrix = [[0 for _ in range(user_count)] for _ in range(user_count())]
for i in range(user_count):
    for j in range(i+1, user_count):
        common_watched = user_movies